## Carregando os pacotes

In [6]:
library(dplyr) # manipulação dos dados
library(ggplot2) # visualização
library(gridExtra) # visualizar vários gráficos juntos
library(tidytext) # mineração de texto
library(wordcloud2) # criação de núvem de palavras

## Lendo os dados

In [2]:
prince_orig <- read.csv("data/prince_raw_data.csv", stringsAsFactors = FALSE)

In [3]:
# Colunas da base de dados
names(prince_orig)

[1] "X"            "text"         "artist"       "song"         "year"        
 [6] "album"        "Release.Date" "US.Pop"       "US.R.B"       "CA"          
[11] "UK"           "IR"           "NL"           "DE"           "AT"          
[16] "FR"           "JP"           "AU"           "NZ"           "peak"

In [4]:
# Primeiras linhas do DataFrame
head(prince_orig)

X  
1  49
2 669
3  78
4 475
5 811
6 478
  text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
1 All 7 and we'll watch them fall\nThey stand in the way of love and we will smoke them all\nWith an intellect and a savior-faire\nNo one in the whole universe will ever compare\nI am yours now and you are mine\nAnd together we'll love through\nAll space and time, so don't cry\nOne day all 7 will die\nAll 7 and we'll watch them fall\nThey stand in the way of love and we will smoke them all\nWith an intellect and a savior-faire\nNo one in the whole universe will ever compare\nI am yours now and you are mine\nAnd together we'll love through\nAll space and time, so don't cry\nOne day all 7 will die\nAnd I saw an Angel come down unto me\nIn her hand she holds the very key\nWords of compassion, words of peace\nAnd in the distance an army's marching feet\n  (1 2 3 4 - 1 2 3 4)\nBut behold, we will watch them fall\nAnd we lay down on the sand of the sea\nAnd before us animosity will stand and decree\nThat we speak not of love only blasphemy\nAnd in the distance, 6 others will curse me\nBut that's alright\n(That's alright)\n4 I will watch them fall\n(1 2 3 4 5 6 7)\nAll 7 and we'll watch them fall\nThey stand in the way of love and we will smoke them all\nWith an intellect and a savior-faire\nNo one in the whole universe will ever compare\nI am yours now and you are mine\nAnd together we'll love through\nAll space and time, so don't cry\nOne day all 7 will die\n(Just how old)\nAnd we will see a plague and a river of blood\nAnd every evil soul will surely die in spite of\nTheir 7 tears, but do not fear\n4 in the distance, 12 souls 

In [7]:
# Selecionando algumas colunas 
prince <- prince_orig  %>% 
    select(lyrics = text, song, year, album, peak, us_pop = US.Pop, us_rnb = US.R.B)

glimpse(prince[139,])

Observations: 1
Variables: 7
$ lyrics <chr> "I just can't believe all the things people say, controversy\n…
$ song   <chr> "controversy"
$ year   <int> 1981
$ album  <chr> "Controversy"
$ peak   <int> 3
$ us_pop <chr> "70"
$ us_rnb <chr> "3"


In [8]:
dim(prince)

[1] 824   7

In [9]:
str(prince[139,]$lyrics, nchar.max = 300)

 chr "I just can't believe all the things people say, controversy\nAm I Black or White? Am I straight or gay? Controversy\nDo I believe in God? Do I believe in me? Controversy\nControversy, controversy\nI can't understand human curiosity, controversy\nWas it good for you? Was I what you w"| __truncated__


## Limpeza básica

In [10]:
# Função responsável por remover as contrações da língua inglesa
fix.contractions <- function(doc) {
    doc <- gsub("won't", "will not", doc)
    doc <- gsub("can't", "can not", doc)
    doc <- gsub("n't", " not", doc)
    doc <- gsub("'ll", "will", doc)
    doc <- gsub("'re", " are", doc)
    doc <- gsub("'ve", " have", doc)
    doc <- gsub("'m", " am", doc)
    doc <- gsub("'d", " would", doc)
    doc <- gsub("'s", "", doc)
    return(doc)
}

In [11]:
prince$lyrics <- sapply(prince$lyrics, fix.contractions)

In [12]:
# Função responsável por remover caracteres especiais
removeSpecialChars <- function(x) gsub("[^a-zA-Z0-9 ]", " ", x)

# Removendo os caracteres especiais
prince$lyrics <- sapply(prince$lyrics, removeSpecialChars)

In [13]:
# Convertendo todas as letras para minúsculo
prince$lyrics <- sapply(prince$lyrics, tolower)

In [15]:
str(prince[139,]$lyrics, nchar.max = 300)

 chr "i just can not believe all the things people say  controversy am i black or white  am i straight or gay  controversy do i believe in god  do i believe in me  controversy controversy  controversy i can not understand human curiosity  controversy was it good for you  was i what you wa"| __truncated__


In [16]:
summary(prince)

    lyrics              song                year         album          
 Length:824         Length:824         Min.   :1978   Length:824        
 Class :character   Class :character   1st Qu.:1989   Class :character  
 Mode  :character   Mode  :character   Median :1996   Mode  :character  
                                       Mean   :1995                     
                                       3rd Qu.:1999                     
                                       Max.   :2015                     
                                       NA's   :495                      
      peak          us_pop             us_rnb         
 Min.   : 0.00   Length:824         Length:824        
 1st Qu.: 2.00   Class :character   Class :character  
 Median : 7.00   Mode  :character   Mode  :character  
 Mean   :15.48                                        
 3rd Qu.:19.00                                        
 Max.   :88.00                                        
 NA's   :751                   

In [17]:
# Anos dos dados
unique(prince$year)

[1] 1992   NA 1982 2006 1994 1998 2009 1999 2002 2004 2010 1987 2007 1988 1985
[16] 1986 1978 2015 1979 1989 1996 1984 1981 1991 2001 1980 1995 1990 1993